# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by reducing the time series to its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center.

Here's a map of the locations of the tidal meters along the River Thames in London.

![](locations.png)

The provided datasets are in the `data` folder in this workspace. For this project, you will work with one of these files, `10-11_London_Bridge.txt`, which contains comma separated values for water levels in the Thames River at the London Bridge. After you've finished the project, you can use your same code to analyze data from the other files (at other spots in the UK where tidal data is collected) if you'd like. 

The TXT file contains data for three variables, described in the table below. 

| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide meter. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |



Analyze Thames River tidal data to track changes in high-tide and low-tide frequency over time.

The data is in the data/10-11_London_Bridge.txt file.

- Load, manipulate, and prepare the data to create new structures and names that might benefit your analysis.

- Find the mean, median, and interquartile range for high- and low-tide data and save them as two separate pandas Series.

- Calculate the annual percentage of days with very high tide levels (90th percentile of high tide days) for each year and store the results as floats in a two-column DataFrame with the index reset.

- Calculate the same percentage for low-tide days (below the 10th percentile) and store the results in the same way.

- Create a dictionary named solution with a summary of your data analysis, with these key-value pairs:

`{high_statistics: high-tide stats, low_statistics: low-tide stats, very_high_ratio: high-tide ratio data, very_low_ratio: low-tide ratio data}`

In [1]:
# We've imported your first Python package for you, along with a function you will need called IQR
import pandas as pd               

def IQR(column): 
    """ Calculates the interquartile range (IQR) for a given DataFrame column using the quantile method """
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

lb = pd.read_csv('data/10-11_London_Bridge.txt')
lb.head()

,Date and time,water level (m ODN),flag,HW=1 or LW=0
0,01/05/1911 15:40:00,3.7130,1,NaN
1,02/05/1911 11:25:00,-2.9415,0,NaN
2,02/05/1911 16:05:00,3.3828,1,NaN
3,03/05/1911 11:50:00,-2.6367,0,NaN
4,03/05/1911 16:55:00,2.9256,1,NaN


### Badanie

In [2]:
lb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115503 entries, 0 to 115502
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Date and time         115503 non-null  object 
 1    water level (m ODN)  115503 non-null  object 
 2    flag                 115503 non-null  int64  
 3    HW=1 or LW=0         0 non-null       float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.5+ MB


In [22]:
df = lb.iloc[:, :3]
df

,Date and time,water level (m ODN),flag
0,01/05/1911 15:40:00,3.7130,1
1,02/05/1911 11:25:00,-2.9415,0
2,02/05/1911 16:05:00,3.3828,1
3,03/05/1911 11:50:00,-2.6367,0
4,03/05/1911 16:55:00,2.9256,1
...,...,...,...
115498,30/12/1995 20:44:00,3.2900,1
115499,31/12/1995 02:32:00,-1.6000,0
115500,31/12/1995 08:59:00,3.2000,1
115501,31/12/1995 15:03:00,-1.8000,0


### Zmiana nazw kolumn i formatu czasu, format water_level i dodanie nowych kolumn

In [23]:
df.columns

Index(['Date and time', ' water level (m ODN)', ' flag'], dtype='object')

In [24]:
df.columns = df.columns.str.strip()

In [25]:
df.rename(columns={
    'Date and time': 'datetime',
    'water level (m ODN)': 'water_level',
    'flag': 'is_high_tide'
    }, inplace=True)

In [27]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%d/%m/%Y %H:%M:%S')

In [28]:
df['water_level'] = df['water_level'].astype(float)

In [29]:
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year

In [30]:
df.head()

,datetime,water_level,is_high_tide,month,year
0,1911-05-01 15:40:00,3.7130,1,5,1911
1,1911-05-02 11:25:00,-2.9415,0,5,1911
2,1911-05-02 16:05:00,3.3828,1,5,1911
3,1911-05-03 11:50:00,-2.6367,0,5,1911
4,1911-05-03 16:55:00,2.9256,1,5,1911


### Pytanie 1

- Find the mean, median, and interquartile range for high- and low-tide data and save them as two separate pandas Series.


In [33]:
high_tide = df[df['is_high_tide'] == 1]
low_tide = df[df['is_high_tide'] == 0]

high_stats = high_tide['water_level'].agg(['mean', 'median', IQR])
low_stats = low_tide['water_level'].agg(['mean', 'median', IQR])

print(high_stats)
print(low_stats)

mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64
mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64


### Pytanie 2

- Calculate the annual percentage of days with very high tide levels (90th percentile of high tide days) for each year and store the results as floats in a two-column DataFrame with the index reset.

In [36]:
total_high_tide_levels_per_year = high_tide.groupby('year')['water_level'].count()
very_high_days = high_tide[high_tide['water_level'] > high_tide['water_level'].quantile(0.90)].groupby('year')['water_level'].count()
very_high_ratio = (very_high_days/total_high_tide_levels_per_year).reset_index()

### Pytanie 3

- Calculate the same percentage for low-tide days (below the 10th percentile) and store the results in the same way.

In [35]:
# Calculate ratio of low tide days
all_low_days = low_tide.groupby('year')['water_level'].count()
very_low_days = low_tide[low_tide['water_level'] < low_tide['water_level'].quantile(0.10)].groupby('year')['water_level'].count()
very_low_ratio = (very_low_days/all_low_days).reset_index()

# Solution

In [ ]:
# Package imports
import pandas as pd                

def IQR(column): 
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

# Load the data from London Bridge
lb = pd.read_csv('data/10-11_London_Bridge.txt') # Comma-separated .txt file

# Take only the first three columns
df = lb.iloc[:, :3]

# Rename columns
df.columns = ['datetime', 'water_level', 'is_high_tide']

# Convert to datetime
df['datetime'] = pd.to_datetime(df['datetime'])

# Convert to float
df['water_level'] = df.water_level.astype(float)

# Create extra month and year columns for easy access
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year

# Filter df for high and low tide
tide_high = df[df['is_high_tide'] == 1]
tide_low = df[df['is_high_tide'] == 0]

# Create summary statistics
high_statistics = tide_high['water_level'].agg(['mean', 'median', IQR])
low_statistics = tide_low['water_level'].agg(['mean', 'median', IQR])

# Calculate ratio of high tide days
all_high_days = tide_high.groupby('year')['water_level'].count()
very_high_days = tide_high[tide_high['water_level'] > tide_high['water_level'].quantile(0.90)].groupby('year')['water_level'].count()
very_high_ratio = (very_high_days/all_high_days).reset_index()

# Calculate ratio of low tide days
all_low_days = tide_low.groupby('year')['water_level'].count()
very_low_days = tide_low[tide_low['water_level'] < tide_low['water_level'].quantile(0.10)].groupby('year')['water_level'].count()
very_low_ratio = (very_low_days/all_low_days).reset_index()

solution = {'high_statistics': high_statistics, 'low_statistics': low_statistics, 'very_high_ratio': very_high_ratio, 'very_low_ratio':very_low_ratio}
print(solution)